In [61]:
# from google.colab import drive
# drive.mount('/content/drive')

In [62]:
%cd /content/drive/MyDrive/Arrhythmia-CNN-master
%ls

/content/drive/MyDrive/Arrhythmia-CNN-master
 Arrhythmia_Classifier.ipynb       'Create Arrhythmia Dataset.py'   save_model/
 Arrhythmia_Classifier.py           data/
'Create Arrhythmia Dataset.ipynb'   README.md


In [63]:
# !pip install 

In [64]:
from __future__ import print_function
import torch
import torch.utils.data
import numpy as np
import pandas as pd
import os

from torch import nn, optim
from torch.utils.data.dataset import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [65]:


is_cuda = False
# num_epochs = 100
num_epochs = 30
batch_size = 10
torch.manual_seed(46)
log_interval = 10
in_channels_ = 1
num_segments_in_record = 100
segment_len = 3600
num_records = 48
num_classes = 16
allow_label_leakage = True

device = torch.device("cuda:0" if is_cuda else "cpu")
# train_ids, test_ids = train_test_split(np.arange(index_set), train_size=.8, random_state=46)
# scaler = MinMaxScaler(feature_range=(0, 1), copy=False)


In [66]:

class CustomDatasetFromCSV(Dataset):
    def __init__(self, data_path, transforms_=None):
        self.df = pd.read_pickle(data_path)
        self.transforms = transforms_

    def __getitem__(self, index):
        row = self.df.iloc[index]
        signal = row['signal']
        target = row['target']
        if self.transforms is not None:
            signal = self.transforms(signal)
        signal = signal.reshape(1, signal.shape[0])
        return signal, target

    def __len__(self):
        return self.df.shape[0]


In [67]:

class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

def basic_layer(in_channels, out_channels, kernel_size, batch_norm=False, max_pool=True, conv_stride=1, padding=0
                , pool_stride=2, pool_size=2):
    layer = nn.Sequential(
        nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=conv_stride,
                  padding=padding),
        nn.ReLU())
    if batch_norm:
        layer = nn.Sequential(
            layer,
            nn.BatchNorm1d(num_features=out_channels))
    if max_pool:
        layer = nn.Sequential(
            layer,
            nn.MaxPool1d(kernel_size=pool_size, stride=pool_stride))

    return layer

class arrhythmia_classifier(nn.Module):
    def __init__(self, in_channels=in_channels_):
        super(arrhythmia_classifier, self).__init__()
        self.cnn = nn.Sequential(
            basic_layer(in_channels=in_channels, out_channels=128, kernel_size=50, batch_norm=True, max_pool=True,
                        conv_stride=3, pool_stride=3),
            basic_layer(in_channels=128, out_channels=32, kernel_size=7, batch_norm=True, max_pool=True,
                        conv_stride=1, pool_stride=2),
            basic_layer(in_channels=32, out_channels=32, kernel_size=10, batch_norm=False, max_pool=False,
                        conv_stride=1),
            basic_layer(in_channels=32, out_channels=128, kernel_size=5, batch_norm=False, max_pool=True,
                        conv_stride=2, pool_stride=2),
            basic_layer(in_channels=128, out_channels=256, kernel_size=15, batch_norm=False, max_pool=True,
                        conv_stride=1, pool_stride=2),
            basic_layer(in_channels=256, out_channels=512, kernel_size=5, batch_norm=False, max_pool=False,
                        conv_stride=1),
            basic_layer(in_channels=512, out_channels=128, kernel_size=3, batch_norm=False, max_pool=False,
                        conv_stride=1),
            Flatten(),
            nn.Linear(in_features=1152, out_features=512),
            nn.ReLU(),
            nn.Dropout(p=.1),
            nn.Linear(in_features=512, out_features=num_classes),
            nn.Softmax()
        )

    def forward(self, x, ex_features=None):
        return self.cnn(x)

def calc_next_len_conv1d(current_len=112500, kernel_size=16, stride=8, padding=0, dilation=1):
    return int(np.floor((current_len + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1))


In [68]:

model = arrhythmia_classifier().to(device).double()
# lr = 0.0003 # learning rate
# lr = 3e-1 # learning rate
lr = 3e-2
# lr = 1e-3
# lr = 3e-4

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
criterion = nn.NLLLoss()


In [69]:

def train(epoch,train_loader):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    # import datetime
    # dt_=datetime.datetime.today().strftime("%Y%m%d-%H%M%S")
    dt_=epoch
    torch.save(model,f'save_model/{dt_}.pt')
    
def test(epoch,test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()

            if batch_idx == 0:
                n = min(data.size(0), 4)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.5f}'.format(test_loss))
    print(f'Learning rate: {optimizer.param_groups[0]["lr"]:.6f}')

def testModel(modelpath_,test_datasets_path,test_loader):
    # model = arrhythmia_classifier().to(device).double()
    # model.load_state_dict(torch.load(modelpath_))
    model=torch.load(modelpath_)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()

            if batch_idx == 0:
                n = min(data.size(0), 4)

    test_loss /= len(test_loader.dataset)
    
    print(f'MODEL：[{modelpath_}]\tTESTDATA：[{test_datasets_path}]')
    print('====> Test set loss: {:.5f}'.format(test_loss))
    print(f'Learning rate: {optimizer.param_groups[0]["lr"]:.6f}')
    return (modelpath_,test_loss)





In [ ]:

# ./data/Arrhythmia_dataset_yystest.pkl
# ./data/Arrhythmia_dataset_yystest_noseed1.pkl
# ./data/Arrhythmia_dataset_yystest_noseed2.pkl
# ./data/Arrhythmia_dataset_yystest_noseed3.pkl
# ./data/Arrhythmia_dataset_yystest_noseed4.pkl
# ./data/Arrhythmia_dataset_yystest_noseed5.pkl

# train_dataset = CustomDatasetFromCSV('./data/Arrhythmia_dataset.pkl')
train_dataset = CustomDatasetFromCSV('./data/Arrhythmia_dataset.pkl')
test_dataset = CustomDatasetFromCSV('./data/Arrhythmia_dataset_yystest.pkl')

# test_dataset = CustomDatasetFromCSV('./data/Arrhythmia_dataset.pkl')
test_datasets_path=[
    "./data/Arrhythmia_dataset_yystest_noseed1.pkl",
    "./data/Arrhythmia_dataset_yystest_noseed2.pkl",
    "./data/Arrhythmia_dataset_yystest_noseed3.pkl",
    "./data/Arrhythmia_dataset_yystest_noseed4.pkl",
    "./data/Arrhythmia_dataset_yystest_noseed5.pkl",
]

if __name__ == "__main__":    
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
    GANKnum_of_iteration = len(train_dataset) // batch_size

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(1, num_epochs + 1):
        train(epoch,train_loader)
        test(epoch,test_loader)
    
    # modellist_=os.listdir("save_model")
    # modelresults_={}
    # for test_dataset_path in test_datasets_path:
    #     test_dataset = CustomDatasetFromCSV(test_dataset_path)
    #     test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    #     for modelpath_ in modellist_:
    #         result_=testModel(f'save_model/{modelpath_}',test_dataset_path,test_loader)
    #         if result_[0] not in modelresults_.keys():
    #           modelresults_[result_[0]]=[]
    #         modelresults_[result_[0]].append(result_[1])
    # for modelname_ in modelresults_:
    #     print(modelname_,end='\t')
    #     for n_ in modelresults_[modelname_]:
    #         print(n_,end='\t')
    #     print(sum(modelresults_[modelname_]))
